In [ ]:
!pip install numpy tensorflow tensorflow-hub google-colab scikit-image matplotlib seaborn scikit-learn tqdm pillow


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from google.colab import drive
from skimage import transform
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.metrics import roc_curve, auc, confusion_matrix, f1_score, classification_report
from sklearn.preprocessing import label_binarize
from tqdm.keras import TqdmCallback

# 1. Data Collection and Preprocessing

# Mount Google Drive
drive.mount('/content/drive')

# Path Variables
train_path = '/content/drive/My Drive/Facial Skin Disease Dataset'

# Define Some Variables
batch_size = 32
img_height = 224  # ResNet50V2 standard input size
img_width = 224   # ResNet50V2 standard input size
seed_train_validation = 1
shuffle_value = True
validation_split = 0.2

# Load Training Images without Augmentation
initial_train_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  validation_split=validation_split,
  subset="training",
  image_size=(img_height, img_width),
  batch_size=batch_size,
  seed=seed_train_validation,
  shuffle=shuffle_value
)

# Extract the class names
class_names = initial_train_ds.class_names
num_classes = len(class_names)
print("The target classes are:", *class_names, sep=", ")

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2)
])

# Load Training Images with Augmentation
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  validation_split=validation_split,
  subset="training",
  image_size=(img_height, img_width),
  batch_size=batch_size,
  seed=seed_train_validation,
  shuffle=shuffle_value
)

# Load Validation Images
val_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  validation_split=validation_split,
  subset="validation",
  image_size=(img_height, img_width),
  batch_size=batch_size,
  seed=seed_train_validation,
  shuffle=shuffle_value
)

# Rescaling the Images for the Model
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Apply Data Augmentation
def augment_image(image, label):
    image = data_augmentation(image)
    return image, label

train_ds = train_ds.map(augment_image)

# Finish the Input Pipeline by Using Buffered Prefetching with Dataset.prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 2. Model Development

# Load Pre-trained ResNet50V2 from TensorFlow Hub
resnet50_v2_fv = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"  # Pre-trained ResNet50V2

# Feature Extractor Layer (ResNet50V2)
feature_extractor_layer = hub.KerasLayer(
    resnet50_v2_fv,
    input_shape=(img_width, img_height, 3),
    trainable=False  # Freeze the pre-trained weights
)

# Build the Model
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(num_classes, activation='softmax')  # Softmax for multi-class classification
])

# Model Summary
model.summary()

# Compile the Model
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)

# 3. Model Training

# Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Define Epochs
NUM_EPOCHS = 10

# Train the Model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=NUM_EPOCHS,
    callbacks=[early_stopping, TqdmCallback(verbose=0)],
    verbose=1
)

# 4. Model Evaluation

# Predictions
val_labels = np.concatenate([y for x, y in val_ds], axis=0)
val_preds = np.concatenate([model.predict(x) for x, y in val_ds], axis=0)
val_preds_class = np.argmax(val_preds, axis=1)

# Confusion Matrix
conf_mat = confusion_matrix(val_labels, val_preds_class)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# ROC Curve
val_labels_bin = label_binarize(val_labels, classes=range(num_classes))
val_preds_bin = tf.nn.softmax(val_preds).numpy()

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(val_labels_bin[:, i], val_preds_bin[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve of class {class_names[i]} (area = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc="lower right")
plt.show()

# F1 Score
f1 = f1_score(val_labels, val_preds_class, average='weighted')
print(f'Weighted F1 Score: {f1:.2f}')

# Classification Report
class_report = classification_report(val_labels, val_preds_class, target_names=class_names)
print('Classification Report:\n', class_report)

# 5. Model Deployment

# Save the Model
model.save('/content/drive/My Drive/save model/facial_skin_disease_resnet50_v2.h5')

# Load the Model
loaded_model = tf.keras.models.load_model('/content/drive/My Drive/save model/facial_skin_disease_resnet50_v2.h5', custom_objects={'KerasLayer': hub.KerasLayer})

# Prediction on New Images
from google.colab import files

# Upload Image
uploaded = files.upload()
uploaded_file_name = list(uploaded.keys())[0]
test_img_path = uploaded_file_name

# Resize Image
test_image = Image.open(test_img_path)
test_image = np.array(test_image).astype('float32') / 255
test_image = transform.resize(test_image, (img_width, img_height, 3))
test_image = np.expand_dims(test_image, axis=0)

# Make Predictions
prediction = loaded_model.predict(test_image)
pred_class = np.argmax(prediction, axis=1)[0]
print(f"The Predicted Class: {class_names[pred_class]}\n")

# View the Uploaded Image
plt.figure(figsize=(3, 3))
test_img = mpimg.imread(test_img_path)
plt.imshow(test_img)
plt.title("Predicted Class: " + class_names[pred_class])
plt.axis('off')
plt.show()
